<a href="https://colab.research.google.com/github/rsfwalters/NEU-OB-MLP/blob/main/Housing_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Housing Data

[write intro about the data]

## 1. Load Data

- [load all the data]
- [show some rows -- pandas like chart]
- [select what are input and the label]


In [ ]:
import pandas as pd
import numpy as np
import torch
import plotly.graph_objects as go

pd.set_option('display.expand_frame_repr', False)  # Show all columns when printing a pandas dataframe

# You can load other datasets from OpenML: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_openml.html

# See details about this dataset: https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset
from sklearn.datasets import fetch_california_housing

# Load the data once as a Pandas DataFrame, so we can inspect the features
housing_frame = fetch_california_housing(as_frame=True).frame

In [ ]:
print(housing_frame)

       MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  Longitude  MedHouseVal
0      8.3252      41.0  6.984127   1.023810       322.0  2.555556     37.88    -122.23        4.526
1      8.3014      21.0  6.238137   0.971880      2401.0  2.109842     37.86    -122.22        3.585
2      7.2574      52.0  8.288136   1.073446       496.0  2.802260     37.85    -122.24        3.521
3      5.6431      52.0  5.817352   1.073059       558.0  2.547945     37.85    -122.25        3.413
4      3.8462      52.0  6.281853   1.081081       565.0  2.181467     37.85    -122.25        3.422
...       ...       ...       ...        ...         ...       ...       ...        ...          ...
20635  1.5603      25.0  5.045455   1.133333       845.0  2.560606     39.48    -121.09        0.781
20636  2.5568      18.0  6.114035   1.315789       356.0  3.122807     39.49    -121.21        0.771
20637  1.7000      17.0  5.205543   1.120092      1007.0  2.325635     39.43    -121.22    

In [ ]:
print(housing_frame.describe())

             MedInc      HouseAge      AveRooms     AveBedrms    Population      AveOccup      Latitude     Longitude   MedHouseVal
count  20640.000000  20640.000000  20640.000000  20640.000000  20640.000000  20640.000000  20640.000000  20640.000000  20640.000000
mean       3.870671     28.639486      5.429000      1.096675   1425.476744      3.070655     35.631861   -119.569704      2.068558
std        1.899822     12.585558      2.474173      0.473911   1132.462122     10.386050      2.135952      2.003532      1.153956
min        0.499900      1.000000      0.846154      0.333333      3.000000      0.692308     32.540000   -124.350000      0.149990
25%        2.563400     18.000000      4.440716      1.006079    787.000000      2.429741     33.930000   -121.800000      1.196000
50%        3.534800     29.000000      5.229129      1.048780   1166.000000      2.818116     34.260000   -118.490000      1.797000
75%        4.743250     37.000000      6.052381      1.099526   1725.000000 

In [ ]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1)

# EXERCISE: Inspect other columns in the data
for idx, col_name in enumerate(["MedInc", "HouseAge"]):
    fig.add_trace(go.Histogram(x=housing_frame[col_name], name=col_name), row=idx+1, col=1)
    fig.update_xaxes(title_text=col_name, row=idx+1, col=1)
fig.show()

In [ ]:
# We can also inspect our target variable (median house price)
fig = go.Figure()
fig.add_trace(go.Histogram(x=housing_frame["MedHouseVal"]))
fig.update_layout(
    title="Histogram of label variable (median house price) ",
    xaxis_title="Median House Price (x $100K)",
    yaxis_title="Counts",
)
fig.show()

In [ ]:
# # EXERCISE: Try uncommenting this block and using normalized_medinc or normalized_houseage as an input

# # Having looked at our data, we see that some features are highly skewed, and are spread on very different ranges.
# # A common normalization method is to describe each value in terms of the mean and standard deviation for the whole group.
# # We can get this representation by subtracting the mean and dividing by the standard deviation.
# # This will help bring different features into a similar scale - Notice how the X-axis of our histogram changes!

# mean_medinc = np.mean(housing_frame["MedInc"])
# stdev_medinc = np.std(housing_frame["MedInc"])
# normalized_medinc = (housing_frame["MedInc"] - mean_medinc) / stdev_medinc

# float_age = np.array(housing_frame["HouseAge"]).astype(np.float32)
# mean_houseage = np.mean(float_age)
# stdev_houseage = np.std(float_age)
# normalized_houseage = (float_age - mean_houseage) / stdev_houseage

# # We can inspect how this changed the distribution of these two input variables.

# from plotly.subplots import make_subplots
# fig = make_subplots(rows=2, cols=1)
# for idx, (values, name) in enumerate([(normalized_medinc, "Normalized MedInc"), (normalized_houseage, "Normalized HouseAge")]):
#     fig.add_trace(go.Histogram(x=values, name=name), row=idx+1, col=1)
#     fig.update_xaxes(title_text=name, row=idx+1, col=1)
# fig.show()

In [ ]:
# # EXERCISE: Try using q_medinc and q_houseage as additional inputs to the model.
# # We can also try adjusting the "constrast" - or moving values around within this range to more evenly spread them out.

# from sklearn.preprocessing import QuantileTransformer
# scaler = QuantileTransformer(output_distribution="uniform")   # EXERCISE: Try adjusting the distribution to "normal" instead!
# q_medinc = scaler.fit_transform(np.expand_dims(housing_frame["MedInc"], -1)).squeeze()
# q_houseage = scaler.fit_transform(np.expand_dims(housing_frame["HouseAge"], -1)).squeeze()

# from plotly.subplots import make_subplots
# fig = make_subplots(rows=2, cols=1)
# for idx, (values, name) in enumerate([(q_medinc, "Normalized MedInc"), (q_houseage, "Normalized HouseAge")]):
#     fig.add_trace(go.Histogram(x=values, name=name), row=idx+1, col=1)
#     fig.update_xaxes(title_text=name, row=idx+1, col=1)
# fig.show()

In [ ]:
# # # EXERCISE: Try using trunc_medinc and trunc_houseage as additional inputs to the model.
# # We can truncate the range of values used, to suppress the effect of outliers. This way, the model will see a more narrow
# # range of values. This approach is simpler than using a percentile transformation, but many items may end up having
# # the minimum or maximum value.

# # EXERCISE - choose a range of values that looks reasonable, based on the original data
# trunc_medinc = np.clip(housing_frame["MedInc"], 0, 8).squeeze()
# trunc_houseage = np.clip(housing_frame["HouseAge"], 0, 35).squeeze()
# from plotly.subplots import make_subplots
# fig = make_subplots(rows=2, cols=1)
# for idx, (values, name) in enumerate([(trunc_medinc, "Truncated MedInc"), (trunc_houseage, "Truncated HouseAge")]):
#     fig.add_trace(go.Histogram(x=values, name=name), row=idx+1, col=1)
#     fig.update_xaxes(title_text=name, row=idx+1, col=1)
# fig.show()

In [ ]:
# Now, let's select a subset of features to use as inputs to the model.
# We'll select columns from the pandas dataframe, and convert them into a numpy array.
# We'll choose median house value ("MedHouseVal") as the target variable.
# We can also engineer additional features based on our intuition about this problem.

ppl_per_bedroom = housing_frame["AveOccup"] / housing_frame["AveBedrms"]
ppl_per_bedroom = np.expand_dims(ppl_per_bedroom, 1)  # To easily combine with our other cols, we'll reshape to (Items, 1)
# ppl_per_room = housing_frame["AveOccup"] / housing_frame["AveRooms"]
# EXERCISE: Try to construct additional features by combining existing features

selected_columns = np.array(housing_frame[["MedInc", "HouseAge"]])
print("Before adding a feature:", selected_columns.shape)
data = np.concatenate([selected_columns, ppl_per_bedroom], axis=1)
print("After adding a feature:", data.shape)
labels = np.array(housing_frame["MedHouseVal"]) 
print("Labels:", labels.shape)

Before adding a feature: (20640, 2)
After adding a feature: (20640, 3)
Labels: (20640,)


In [ ]:
# Split the data into a train, validation, and test set.
# We'll try to extract as much information as we can from the train set.
# Periodically, we can check the validation set to see how we're doing (like a practice quiz)
# Only once, at the very end, we check our performance on the test set.

N_items = data.shape[0]

# EXERCISE: Try adjusting the fraction of data used for training, validation, and test
test_fraction = 0.1
val_fraction = 0.1
shuffled_indices = np.random.permutation(N_items)   # EXERCISE: Try not shuffling the data
N_test = int(test_fraction * N_items)
N_val = int(val_fraction * N_items)

shuffled_data = data[shuffled_indices]
shuffled_labels = labels[shuffled_indices]
test_data, test_labels = shuffled_data[:N_test], shuffled_labels[:N_test]
val_data, val_labels = shuffled_data[N_test:N_test+N_val], shuffled_labels[N_test:N_test+N_val]
train_data, train_labels = shuffled_data[N_test+N_val:], shuffled_labels[N_test+N_val:]

print("Training set:", train_data.shape, train_labels.shape)
print("Validation set:", val_data.shape, val_labels.shape)
print("Test set:", test_data.shape, test_labels.shape)

Training set: (16512, 3) (16512,)
Validation set: (2064, 3) (2064,)
Test set: (2064, 3) (2064,)


In [ ]:
# We'll use 32-bit floating point values, which should be plenty of precision for our purposes.
# Note that the data type of our data must match the data type of our model layers later on.

train_data, train_labels = train_data.astype("float32"), train_labels.astype("float32")
val_data, val_labels = val_data.astype("float32"), val_labels.astype("float32")
test_data, test_labels = test_data.astype("float32"), test_labels.astype("float32")

In [ ]:
# Convert the data for use in pytorch
from torch.utils.data import TensorDataset, DataLoader

train_dataloader = DataLoader(
    TensorDataset(torch.as_tensor(train_data), torch.as_tensor(train_labels)),
    # EXERCISE: Try varying the training batch size
    batch_size=32,     # How many items should we grab for each gradient descent step
    pin_memory=True,   # Helps transfer data to/from GPU faster
    shuffle=True,      # In each epoch, we shuffle the training data
)

val_dataloader = DataLoader(
    TensorDataset(torch.as_tensor(val_data), torch.as_tensor(val_labels)),
    batch_size=32,     # How many items should we grab for each gradient descent step
    pin_memory=True,   # Helps transfer data to/from GPU faster
    shuffle=False,     
)

test_dataloader = DataLoader(
    TensorDataset(torch.as_tensor(test_data), torch.as_tensor(test_labels)),
    batch_size=32,     # How many items should we grab for each gradient descent step
    pin_memory=True,   # Helps transfer data to/from GPU faster
    shuffle=False,     
)

## 2. Define Model

- highlight choices of layer types
- widths
- depth
- normalization

Keep as simple as possible
Comment out options and add comments pointing to things to play with

In [ ]:
from torch import nn
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import MultiplicativeLR
import torch.nn.functional as F

class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(3, 32)  # NOTE - we must match our input dimension with the number of input variables used
        # EXERCISE: Try adding or removing a layer
        self.layer2 = nn.Linear(32, 32) # EXERCISE: Try varying the width of the hidden layers
        self.layer3 = nn.Linear(32, 1)

    def forward(self, x):
        # Here x is a batch of input data
        h1 = self.layer1(x)
        h1 = F.relu(h1)
        h2 = self.layer2(h1)
        # EXERCISE: Try adding skip-connections. For ex, the output of layer 2 could be added to the output of layer 3
        # h2 = F.relu(h2) + h1
        h2 = F.relu(h2)
        h3 = self.layer3(h2)
        # EXERCISE - Try applying an activation at the end to restrict the output to a reasonable range
        # For example, try using F.sigmoid (https://pytorch.org/docs/stable/generated/torch.nn.functional.sigmoid.html#torch.nn.functional.sigmoid)
        # and then multiplying
        return h3

model = MyModel()
model = model.to("cuda")  # Send the model to the GPU

# We will use the optimizer to adjust the model parameters after predicting each batch
optim = Adam(model.parameters(), lr=1e-4, betas=(0.9, 0.999), weight_decay=1e-3)   
# EXERCISE: Try using a different optimizer, or varying parameters like learning rate, momentum, and weight decay
# optim = SGD(model.parameters(), lr=1e-3, momentum=1e-5, weight_decay=1e-5)

# After each epoch, we can reduce the learning rate. 
sched = MultiplicativeLR(optim, lr_lambda=lambda epoch: 0.95)

## 3. Train and Evaluate

- qualitative and quantitative metrics
- show worst predictions
- examples of outputs
- training and validation 

- learning rate
- loss functions 
- regularization

Note - if you adjust the training procedure, be sure to re-initialize the model by re-running that cell above. Otherwise, your model would not be starting "from scratch"; it would start training from the point where it left off.


In [ ]:
from tqdm.notebook import tqdm, trange

In [ ]:
# We'll keep note of our training loss at every batch, and our validation loss at every epoch
train_loss_tracking, val_loss_tracking = [], [] 
test_preds = []
epochs = 2

for epoch in trange(epochs, desc="epochs", leave=True, position=0):
    # Perform an epoch of training
    model.train()  # Set the model into training mode. This affects some layer behavior, such as dropout and batchnorm
    for batch_data, batch_labels in tqdm(train_dataloader, desc="batches", leave=False, position=1):
        batch_data = batch_data.to("cuda")
        batch_labels = batch_labels.to("cuda")

        predictions = model(batch_data).squeeze()
        loss = F.mse_loss(predictions, batch_labels)
        optim.zero_grad()
        loss.backward()
        optim.step()

        train_loss_tracking.append(loss.item())

    # Check our performance on the validation set
    with torch.no_grad():
        avg_val_loss = 0.0
        model.eval()
        for batch_data, batch_labels in tqdm(val_dataloader, desc="batches", leave=False, position=1):
            batch_data = batch_data.to("cuda")
            batch_labels = batch_labels.to("cuda")

            predictions = model(batch_data).squeeze()
            avg_val_loss += F.mse_loss(predictions, batch_labels, reduction="sum")  # Sum here, since we'll average ourselves
        avg_val_loss /= len(val_dataloader.dataset)    
        val_loss_tracking.append(avg_val_loss.item())
    sched.step()  # Adjust our learning rate after every epoch


# Finally at the very end, check our performance on the test set
with torch.no_grad():
    model.eval()   # Set the model into evaluation mode.
    avg_test_loss = 0.0
    for batch_data, batch_labels in tqdm(test_dataloader):
        batch_data = batch_data.to("cuda")
        batch_labels = batch_labels.to("cuda")
        predictions = model(batch_data).squeeze()
        test_preds.extend(list(predictions.cpu().numpy()))
        avg_test_loss += F.mse_loss(predictions, batch_labels, reduction="sum")
    avg_test_loss /= len(test_dataloader.dataset)

epochs:   0%|          | 0/2 [00:00<?, ?it/s]

batches:   0%|          | 0/516 [00:00<?, ?it/s]

batches:   0%|          | 0/65 [00:00<?, ?it/s]

batches:   0%|          | 0/516 [00:00<?, ?it/s]

batches:   0%|          | 0/65 [00:00<?, ?it/s]

  0%|          | 0/65 [00:00<?, ?it/s]

In [ ]:
print("Average test loss:", round(avg_test_loss.item(), 3))

Average test loss: 0.824


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=test_labels, y=test_preds, mode="markers"))
fig.update_layout(
    title="Compare true and predicted values",
    xaxis_title="True price",
    yaxis_title="Predicted price",
)
fig.show()

In [ ]:

N = 20

mean_x3 = np.mean(ppl_per_bedroom).astype(np.float32)

dummy_data = []
for x1 in np.linspace(np.min(test_data[:, 0]), np.max(test_data[:, 0]), N, dtype=np.float32):
    for x2 in np.linspace(np.min(test_data[:, 1]), np.max(test_data[:, 1]), N,dtype=np.float32):
            dummy_data.append((x1, x2, mean_x3))

dummy_data = np.array(dummy_data)
with torch.no_grad():
    model.eval()
    dummy_preds = model(torch.as_tensor(dummy_data).to("cuda")).cpu().numpy().squeeze()

fig = go.Figure()
fig.add_trace(go.Contour(x=dummy_data[:, 0], z=dummy_data[:, 1], y=dummy_preds))
fig.show()

In [ ]:
from plotly.subplots import make_subplots

fig = go.Figure()
fig.update_layout(
    title="Training and Validation loss",
    xaxis_title="Batch number",
    yaxis_title="Loss value",
)
fig.add_trace(go.Scatter(x=np.arange(len(train_loss_tracking)), y=train_loss_tracking, name="Training"))

# Each item in the validation loss list was measured after an epoch.
# Each epoch consists of multiple batches
batches_per_epoch = len(train_dataloader)
x = np.arange(1, len(val_loss_tracking)+1) * batches_per_epoch
fig.add_trace(go.Scatter(x=x, y=val_loss_tracking, name="Validation"))
fig.show()
